# Collect house price data

The best way to collect house price data is using [Zillow API](https://www.zillow.com/howto/api/APIOverview.htm).

This notebook is an academic demonstration of how to crawl information from websites. The data used in this demo is a single page manually downloaded from [Trulia](https://www.trulia.com) and then uploaded to a dummy website hosted by AWS S3.

## Import libraries 

In [ ]:
import pandas
import configparser
import psycopg2

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['db']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

In [ ]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

## Create a table in database

In [ ]:
# replace the schema and table name to your schema and table name if necessary
table_sql = """
            CREATE TABLE IF NOT EXISTS house
            (

                price integer,
                bed integer,
                bath integer,
                area integer,
                address VARCHAR(200),
                PRIMARY KEY(address)
            );

            """

In [ ]:
# conn.rollback()
# table_sql="drop table if exists house"

In [ ]:
cur.execute(table_sql)
conn.commit()

## Define the URL

Fill in the S3 website URL to the `url` variable.

In [ ]:
url = 'https://github.com/xbwei/Data-Mining-on-Social-Media/blob/master/Minning%20House%20Price%20from%20Trulia/Collect_house_price_data.ipynb'

## Collect the research results 

In [ ]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
# print(html_data.decode('utf-8'))

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
# print (soup)

## Insert the records into database

In [ ]:
for ul in soup.find_all('ul'):
    for li_class in ul.find_all('li'):
        try:
            for price_div in li_class.find_all('div',{'data-testid':'property-price'}):
                price =int(price_div.text.replace('$','').replace(",",""))
            for bed_div in li_class.find_all('div', {'data-testid':'property-beds'}):
                bed= int(bed_div.text.replace('bd','').replace(",",""))
            for bath_div in li_class.find_all('div',{'data-testid':'property-baths'}):
                bath =int(bath_div.text.replace('ba','').replace(",",""))
            for area_div in li_class.find_all('div',{'data-testid':'property-floorSpace'}):
                area=int(area_div.text.split('sqft')[0].replace(",",""))
            for address_div in li_class.find_all('div',{'data-testid':'property-address'}):
                address =address_div.text
            try:
                sql_insert = """
                            insert into house(price,bed,bath,area,address)
                            values('{}','{}','{}','{}','{}')            
                            """.format(price,bed,bath,area,address)

                cur.execute(sql_insert)
                conn.commit()
            except:
                conn.rollback()
        except:
            pass

## Query the table

In [ ]:
df = pandas.read_sql_query('select * from house ', conn)
df[:]

## Basic statistics 

In [ ]:
df.describe()

## Price distribution

In [ ]:
df['price'].hist()

In [ ]:
df.plot.scatter(x='bed',y='bath')

In [ ]:
df['price'].mean()

In [ ]:
sql =
"""

select  avg(price/area) as avg_unit_price from house


"""